## Convert SkHASH format focal mechanisms to STRESSINVERSE format .dat file

In [29]:
import os
import numpy as np
import pandas as pd

project_dir = '../'

In [30]:
# read focal mechanism data
my_result_dir = '../results/foc_mech'
skhash_df = pd.read_csv(f'{my_result_dir}/ALL_Pyrocko_varified_OUT.csv')

# drop low quality mechanisms [D]
skhash_df = skhash_df[skhash_df.quality != 'D']

## for clusters of earthquakes, create subset dataframes of the eq catalog

In [31]:
eq_cat_file = f'{project_dir}/data/eq_data/1_eq_catalogs/' + 'usgs_eq_above_slab2_all.csv'
eq_cat_df = pd.read_csv(eq_cat_file, parse_dates=['time'])

# get clusters from [0]_Map_plot.ipynb
clusters = np.array([
    [1, -124.6, -124.22, 40.2, 40.37],
    [2, -124.21, -124.05, 40.19, 40.27],
    [3, -124.0, -123.6, 40.0, 40.2],
    [4, -123.3, -123, 40.05, 40.25],
    [5, -123.6, -123.4, 40.4, 40.6],
    [6, -124.27, -123.95, 40.52, 40.65],
])

# create a subset dataframe for each cluster
cluster_dfs = {}
for cluster in clusters:
    cluster_id, min_lon, max_lon, min_lat, max_lat = cluster
    cluster_dfs[cluster_id] = eq_cat_df[
        (eq_cat_df['longitude'] >= min_lon) & (eq_cat_df['longitude'] <= max_lon) &
        (eq_cat_df['latitude'] >= min_lat) & (eq_cat_df['latitude'] <= max_lat)
    ]
cluster_dfs[6].head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,slab_depth
34,2023-05-19 06:06:45.570000+00:00,40.598167,-124.080833,16.11,2.05,md,21.0,49.0,0.11600,0.13,...,"6km NNE of Hydesville, CA",earthquake,0.30,0.81,0.203,21.0,reviewed,nc,nc,17.456727
35,2023-05-18 11:27:14.260000+00:00,40.586667,-124.111833,16.12,2.19,md,26.0,54.0,0.10090,0.13,...,"4km ESE of Fortuna, CA",earthquake,0.27,0.64,0.285,26.0,reviewed,nc,nc,17.050866
36,2023-05-13 07:12:42.790000+00:00,40.553667,-124.240833,15.05,2.16,md,24.0,99.0,0.02842,0.14,...,"3km SE of Ferndale, CA",earthquake,0.51,0.72,0.138,14.0,reviewed,nc,nc,15.446382
46,2023-02-22 15:48:21.610000+00:00,40.559500,-124.220333,15.63,3.07,ml,34.0,70.0,0.03697,0.11,...,"4km ESE of Ferndale, CA",earthquake,0.34,0.46,0.241,23.0,reviewed,nc,nc,15.691121
50,2023-01-25 01:47:20.640000+00:00,40.571333,-124.233833,9.23,2.53,md,32.0,83.0,0.02323,0.14,...,"3km ESE of Ferndale, CA",earthquake,0.22,0.41,0.123,38.0,reviewed,nc,nc,15.495134


In [37]:
cluster_df = cluster_dfs[5]
skhash_cluster_df = skhash_df[skhash_df.event_id.isin(cluster_df.id)]
skhash_cluster_df.head()

,event_id,strike,dip,rake,quality,fault_plane_uncertainty,aux_plane_uncertainty,num_p_pol,num_sp_ratios,polarity_misfit,prob_mech,sta_distribution_ratio,sp_misfit,mult_solution_flag
24,nc71255501,27.4141,48.4668,-88.4533,C,35.9151,34.5383,20,0,15.2,61.0,73.8,0.0,False


In [ ]:
## sample output data
"""
% West Bohemia mechanisms
%  strike          dip             rake  
   1.5264276e+02   6.2186955e+01  -3.6060595e+01
"""

# write to file in stressinverse format
out_dir = 'Stressinverse_1.1.3/Data'
with open(f'{out_dir}/MTJ_manual_mechanisms.dat', 'w') as f:
    f.write('% MTJ manual mechanisms\n')
    f.write('%  strike          dip             rake  \n')
    for i, row in skhash_df.iterrows():
        strike = f'{row.strike:.6e}'
        dip = f'{row.dip:.6e}'
        rake = f'{row.rake:.6e}' if row.rake < 0 else f' {row.rake:.6e}'
        f.write(f'   {strike:<16}{dip:<16}{rake}\n')

## Change the input parameters file

In [25]:
id = 'MTJ_C1'
input_file = f'../Data/{id}_manual_mechanisms.dat'
output_file = f'../Output/{id}_manual_stress_output'
principal_mechanisms_file = f'../Output/{id}_manual_principal_mechanisms'
N_noise_realizations = 100
fig_dir = f'../Figures/{id}'

# read Input_parameters.py and change the essential parameters
with open('Stressinverse_1.1.3/Programs_PYTHON/Input_parameters.py', 'r') as f:
    lines = f.readlines()
    print(lines[16].strip().split())

    with open('Stressinverse_1.1.3/Programs_PYTHON/Input_parameters_test.py', 'w') as f:
        for i, line in enumerate(lines):
            if len(line.strip().split()) > 0:
                if line.strip().split()[0] == 'input_file':
                    f.write(f"input_file = r'{input_file}'\n")
                elif line.strip().split()[0] == 'output_file':
                    f.write(f"output_file = r'{output_file}'\n")
                elif line.strip().split()[0] == 'principal_mechanisms_file':
                    f.write(f"principal_mechanisms_file = r'{principal_mechanisms_file}'\n")
                elif line.strip().split()[0] == 'N_noise_realizations':
                    f.write(f"N_noise_realizations = {N_noise_realizations}\n")
                elif line.strip().split()[0] == 'fig_dir':
                    f.write(f"fig_dir = r'{fig_dir}'\n")
                else:
                    f.write(line)
                
            else:
                f.write(line) # write empty lines

['input_file', '=', "r'../Data/MTJ_manual_mechanisms.dat'"]


In [29]:
import scipy.io as sio

mat = sio.loadmat('Stressinverse_1.1.3/Output/MTJ_manual_stress_output.mat')
print(mat)

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Wed Apr 24 11:00:27 2024', '__version__': '1.0', '__globals__': [], 'sigma_1': array([[(array([[344.4537399]]), array([[67.19746525]]))]],
      dtype=[('azimuth', 'O'), ('plunge', 'O')]), 'sigma_2': array([[(array([[184.56301988]]), array([[21.54333121]]))]],
      dtype=[('azimuth', 'O'), ('plunge', 'O')]), 'sigma_3': array([[(array([[91.73668165]]), array([[7.11942324]]))]],
      dtype=[('azimuth', 'O'), ('plunge', 'O')]), 'shape_ratio': array([[0.30301264]]), 'mechanisms': array([[(array([[ 37.10343419,  22.51450725, 341.81865708, 220.91816752,
                338.6172    ,  15.51614659, 200.097     , 177.6127    ,
                115.2292    , 340.6525    , 175.2424    , 274.56029777,
                343.2857    , 176.6176    , 325.9181    , 212.8225    ,
                122.02689206, 193.58383525,   2.69749167,  44.70192874,
                345.32608788, 210.9786    , 219.77626758, 176.4431    ,
                20